In [ ]:
# all import statements needed
import os
import json
import requests
import shutil
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
import pandas as pd

In [ ]:
# get planet api key from .env
PLANET_API_KEY = os.getenv('PLANET_API_KEY')

In [ ]:
class Region:
    def __init__(self, name, region, dates, cloud_cover, item_type, asset_types, search_result = None, images=[], active_requests = dict(), activated = set()):
        self.name = name
        # polygon json that contains coordinates of region of intrest
        self.region = region
        # locations that overlap region of interest
        self.location = {
            "type": "GeometryFilter",
            "field_name": "geometry",
            "config": region
        }
        # json that contains dates of interest
        self.dates = dates
        # json that has cloud cover threshol
        self.cloud_cover = cloud_cover
        # json that combines all previous filters
        self.filters = {
            "type": "AndFilter",
            "config": [self.location, dates, cloud_cover]
        }
        # item-type of images desired
        self.item_type = item_type
        # desired asset types
        self.asset_types = asset_types.split()
        # will contain post of search results
        self.search_result = search_result
        # will contains image id of all images
        self.images = images
        # will contain request for each image
        self.active_requests = {asset_type:[] for asset_type in self.asset_types}
        # will contain images that still are activating
        self.activated = activated

In [ ]:
df = pd.read_csv('locations.csv')
print(json.loads(df['Dates'][2]))
# print(df)

In [ ]:
regions = []
# create objects of regions
for i in df.index:
    regions.append(Region(df['Region Name'][i], json.loads(df['Polygon Coordinates'][i]), json.loads(df['Dates'][i]), json.loads(df['Cloud Coverage'][i]), df['Item Type'][i], df['Asset Type'][i]))

In [ ]:
# set the item type that we are insterested
for region in regions:
    # create a search with filters that we want for each region
    search = {
    "item_types":['PSScene'],
    "filter":region.filters
    }
    # post the request using our PLANET_API_KEY to authenticate
    search_result = \
        requests.post(
            'https://api.planet.com/data/v1/quick-search',
            auth=HTTPBasicAuth(PLANET_API_KEY, ''),
            json=search
        )
    # add the request to region object
    region.search_result = search_result
    region.images = [feature['id'] for feature in search_result.json()['features']]
    

In [ ]:
# download all data for all regions

# for each region in the csv file
for region in regions:
    # for each asset_type that we want from each region
    for asset_type in region.asset_types:
        for i, image in enumerate(region.images):
            image_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(region.item_type, image)
            # create a get request with image url
            result = \
                requests.get(
                    image_url,
                    auth=HTTPBasicAuth(PLANET_API_KEY, '')
                )
            # if the result has a an asset type then that we are looking for
            if asset_type in result.json().keys():
                links = result.json()[asset_type]["_links"]
                self_link = links["_self"]
                activation_link = links["activate"]
                # activate that request
                activate_result = \
                    requests.get(
                        activation_link,
                        auth=HTTPBasicAuth(PLANET_API_KEY, '')
                    )
                # will give us the atatus of the request
                activation_status_result = \
                    requests.get(
                        self_link,
                        auth=HTTPBasicAuth(PLANET_API_KEY, '')
                    )
                # append the requests
                region.active_requests[asset_type].append(activation_status_result)
                region.activated.add(activation_status_result)
        # in order to not have a connection time out we must first download all of the images before we can proceed
        while region.activated:
            for i, request in enumerate(region.active_requests[asset_type]):
                # checks to see if request is ready
                if request in region.activated and request.json()["status"] == 'active':
                    # gets url of image
                    url = request.json()["location"]
                    # name of file
                    name = region.name +" " + asset_type +"/image" + str(i) +".tif"
                    # creates a directory in the working space if it doesn't already exist
                    if not os.path.exists(region.name + " " + asset_type):
                        os.makedirs(region.name + " " + asset_type)
                    download = requests.get(url, stream=True)
                    # if we don't do this then the file size will be 0
                    download.raw.decode_content = True
                    # download the file
                    with open(name, "wb") as f:
                        shutil.copyfileobj(download.raw, f)
                    # the status of this request was active so remove from the set
                    region.activated.remove(request)
        region.activated.clear()